In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


# -*- coding: utf-8 -*-
"""
Created on Sun Mar  6 15:55:40 2022

@author: 99259
"""
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import package_dataprocess.Nilm_classes
import numpy as np
import keras
# from nilm_ukdale import preprocess
from keras import Model
from keras.models import Sequential
import os
import gc
import pandas as pd
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import PIL.ImageOps
from keras import Model
from keras import layers
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import MaxPooling3D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import GRU, TimeDistributed, Bidirectional
from keras.layers import Embedding
import matplotlib.pyplot as plt

# In[2]:


# 电器id定义
fridge_id = 0
tv_id = 1
kettle_id = 2
microwave_id = 3
washerdryer_id = 4

# 电器开启状态阈值
fridge_threshold = 15
tv_threshold = 15
kettle_threshold = 15
microwave_threshold = 15
washerdryer_threshold = 15

# dp = package_dataprocess.Nilm_classes.Nilm_dataprocess(path_csv="DATA/vicurve_name.xlsx",path_img="DATA/vicurve32x32/")
sum_data_num = np.zeros(100800)

filenames = os.listdir(r'DATA\UKData_by_hour\week3\LowFreq')
# print(filenames)


# In[3]:


fridge_data = np.zeros(100800)
television_data = np.zeros(100800)
kettle_data = np.zeros(100800)
microwave_data = np.zeros(100800)
washerdryer_data = np.zeros(100800)

# In[4]:


for time in range(7 * 24):
    temp = np.zeros(100800)
    for i in filenames:
        if (i[:10] == str(1453075200 + 3600 * time)):#换周的话记得换一下时间戳喔
            name = i[11:-4]
            name_data = np.load('DATA/UKData_by_hour/week3/LowFreq/' + i)
            name_data = np.array(name_data)
            # print(name_data.shape)
            if len(name_data) > 600:
                name_data = name_data[:600]
            elif len(name_data) < 600:
                name_data = np.pad(name_data, (0, 600 - len(name_data)), 'constant', constant_values=(0, 0))
            name_data = np.where(name_data > 3, name_data, 0)  # 底噪3w

            if (name == 'Fridge freezer0'):
                fridge_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Television0'):
                television_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Kettle0'):
                kettle_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Microwave0'):
                microwave_data[time * 600:(time + 1) * 600] = name_data
            if (name == 'Washer dryer0'):
                washerdryer_data[time * 600:(time + 1) * 600] = name_data

            temp[time * 600:(time + 1) * 600] = name_data
            sum_data_num = sum_data_num + temp
            # print(sum_data_num.shape)
            # sum_data_num = sum_data_num[time*600:(time + 1) * 600] + name_data

# In[5]:


dp = package_dataprocess.Nilm_classes.Nilm_dataprocess(path_csv="DATA/UKData_by_hour/week3/vicurve_name.xlsx",
                                                       path_img="DATA/UKData_by_hour/week3/vicurve32x32/")

# In[6]:


print(np.array(fridge_data).shape)

# In[7]:


fridge_data = fridge_data.reshape(fridge_data.shape[0], 1)
television_data = television_data.reshape(television_data.shape[0], 1)
kettle_data = kettle_data.reshape(kettle_data.shape[0], 1)
microwave_data = microwave_data.reshape(microwave_data.shape[0], 1)
washerdryer_data = washerdryer_data.reshape(washerdryer_data.shape[0], 1)

# fridge_data = np.load('DATA/UKData_2/Fridge freezer0.npy')
# fridge_data = fridge_data[:100800]
# fridge_data = np.where(fridge_data > dp.air_threshold,fridge_data,0)
#
# television_data = np.load('DATA/UKData_2/Television0.npy')
# television_data = television_data[:100800]
# television_data = np.where(television_data > dp.air_threshold, television_data,0)
#
# kettle_data = np.load('DATA/UKData_2/Kettle0.npy')
# kettle_data = kettle_data[:100800]
# kettle_data = np.where(kettle_data > dp.air_threshold, kettle_data,0)
#
# microwave_data = np.load('DATA/UKData_2/Microwave0.npy')
# microwave_data = microwave_data[:100800]
# microwave_data = np.where(microwave_data > dp.air_threshold, microwave_data,0)
#
# washerdryer_data = np.load('DATA/UKData_2/Washer dryer0.npy')
# washerdryer_data = washerdryer_data[:100800]
# washerdryer_data = np.where(washerdryer_data > dp.air_threshold, washerdryer_data,0)
#
# Fan_data = np.load('DATA/UKData_2/Fan0.npy')
# Fan_data = Fan_data[:100800]
# Fan_data = np.where(fridge_data > dp.air_threshold,Fan_data,0)
#
#

# In[8]:


fridge_labels = dp.create_label(fridge_data, 0, fridge_threshold)
tv_labels = dp.create_label(television_data, 3, tv_threshold)
kettle_labels = dp.create_label(kettle_data, 1, kettle_threshold)
microwave_labels = dp.create_label(microwave_data, 2, microwave_threshold)
washerdryer_labels = dp.create_label(washerdryer_data, 4, washerdryer_threshold)

sum_label = fridge_labels + tv_labels + kettle_labels + microwave_labels + washerdryer_labels
print(sum_label.shape)
mean = sum_data_num[:100800].mean(axis=0)
sum_data = sum_data_num - mean
std = sum_data[:100800].std(axis=0)
sum_data /= std

# In[9]:


sum_data = np.array(sum_data)
print(sum_data.shape)

sum_data = sum_data.reshape(sum_data.shape[0], 1)
print(sum_data.shape)

# In[10]:





# In[11]:


from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import concatenate
from keras.layers import InputLayer

from keras.models import load_model
model = load_model('CNNRNNMODEL_CONCA_continue.hdf5')

filepath = 'CNNRNNMODEL_CONCA_continue.hdf5'

一切都是命运石之门的选择！
1453075200Kettle0
1453078800Kettle0
1453082400Kettle0
1453086000Kettle0
1453089600Kettle0
1453093200Kettle0
1453096800Kettle0
1453100400Kettle0
1453104000Kettle0
1453107600Kettle0
1453111200Kettle0
1453114800Kettle0
1453118400Kettle0
1453122000Kettle0
1453125600Kettle0
1453129200Kettle0
1453132800Kettle0
1453136400Kettle0
1453140000Kettle0
1453143600Kettle0
1453147200Kettle0
1453150800Kettle0
1453154400Kettle0
1453158000Kettle0
1453161600Kettle0
1453165200Kettle0
1453168800Kettle0
1453172400Kettle0
1453176000Kettle0
1453179600Kettle0
1453183200Kettle0
1453186800Kettle0
1453190400Kettle0
1453194000Kettle0
1453197600Kettle0
1453201200Kettle0
1453204800Kettle0
1453208400Kettle0
1453212000Kettle0
1453215600Kettle0
1453219200Kettle0
1453222800Kettle0
1453226400Kettle0
1453230000Kettle0
1453233600Kettle0
1453237200Kettle0
1453240800Kettle0
1453244400Kettle0
1453248000Kettle0
1453251600Kettle0
1453255200Kettle0
1453258800Kettle0
1453262400Kettle0
1453266000Kettle0
1453269600Kett

In [2]:
sumlabel = sum_label.T

In [3]:
fridge = 0
kettle = 0
microwave = 0
tv = 0
washerdryer = 0

In [4]:
for i in range(100800):
    if(sumlabel[0][i]==1):
        fridge+=1
    if(sumlabel[1][i]==1):
        kettle+=1
    if(sumlabel[2][i]==1):
        microwave+=1
    if(sumlabel[3][i]==1):
        tv+=1
    if(sumlabel[4][i]==1):
        washerdryer+=1

In [5]:
print(fridge/100800)
print(kettle/100800)
print(microwave/100800)
print(tv/100800)
print(washerdryer/100800)

0.36989087301587303
0.00744047619047619
0.010793650793650795
0.15930555555555556
0.027043650793650793
